In [1]:
import numpy as np
import scanpy as sc
import seaborn as sns
import pandas as pd 
# import scvi
from scipy.stats import median_abs_deviation
import matplotlib.pyplot as plt
import os

# Set up the environment
sc.settings.verbosity = 0
# sc.settings.set_figure_params(dpi=120)
sc.settings.set_figure_params(
    dpi=120,
    facecolor="white",
    frameon=False,
)

In [ ]:
# Lectura de los datos
l_adatas = [x for x in os.listdir('./raw_files/') if x.endswith('.h5')]
# Seleccionamos los que "raw_gene_bc_matrices" en su nombre
l_adatas = [x for x in l_adatas if 'raw_gene_bc_matrices' in x]
l_adatas

In [2]:
def load_it(l_adata):
    samp = l_adata.split('_')[0]
    dx = l_adata.split('_')[1]
    l_adata = sc.read_10x_h5('./raw_files/' + l_adata)  # Para archivos 10x h5
    l_adata.obs['Patient'] = samp
    l_adata.obs['DX'] = dx
    l_adata.obs['Sample'] = l_adata.obs['Patient'] + '_' + l_adata.obs['DX']
    l_adata.obs.index = l_adata.obs.index + '-' + samp + '_' + dx
    return l_adata

In [ ]:
l_adatas = [load_it(ad) for ad in l_adatas]

for adata in l_adatas:
    adata.var_names_make_unique()
    print(f"Unique var names for {adata.obs['Sample'][0]}: {adata.var_names.is_unique}")

l_adatas

In [ ]:
for ad in l_adatas: 
    print(f"{ad.obs['Patient'][0]}_{ad.obs['DX'][0]}: {ad.n_obs} cells, {ad.n_vars} genes, total counts: {ad.X.sum()}")

# Removemos células que tengan un porcentaje mayor al 5% de DNA mitocondrial

In [ ]:
# Función para filtrar los genes y las células
def qc(l_adata):
    sc.pp.filter_cells(l_adata, min_genes = 200) 
    l_adata.var["mt"] = l_adata.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(l_adata, qc_vars="mt", inplace=True, percent_top=[5], log1p=True)
    remove = ['total_counts_mt', 'log1p_total_counts_mt']
    l_adata.obs = l_adata.obs[[x for x in l_adata.obs.columns if x not in remove]]
    return l_adata

In [ ]:
l_adatas = [qc(ad) for ad in l_adatas]

In [ ]:
for adata in l_adatas:
    print(f"After QC: {adata.obs['Patient'][0]}_{adata.obs['DX'][0]}: {adata.n_obs} cells, {adata.n_vars} genes, total counts: {adata.X.sum()}")

In [ ]:
# Violin plots de QC para ver cómo se distribuyen los datos
import plotly.express as px

adata = l_adatas[0]
fig = px.violin(adata.obs, y="pct_counts_mt", x="Sample", color="DX", box=True, points="all")
fig.update_traces(meanline_visible=True)
fig.update_layout(title=f"Log total counts per cell for {adata.obs['Sample'][0]}")
fig.show()

In [ ]:
t_df = pd.concat([ad.obs for ad in l_adatas], axis=0)
t_df = t_df.sort_values(by='Sample')
t_df

In [ ]:
value = "pct_counts_mt"
#value = "n_genes"
#value = 'pct_counts_in_top_20_genes'
#value = "log1p_total_counts"

sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

g = sns.FacetGrid(t_df, row="Sample", hue="Sample", aspect=10, height=1, palette="tab20")

g.map(sns.kdeplot, value, clip_on=False, fill=True, alpha=1, linewidth=1.5)
g.map(sns.kdeplot, value, clip_on=False, color="w", lw=2)

g.map(plt.axhline, y=0, lw=2, clip_on=False)

def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

g.map(label, value)

g.figure.subplots_adjust(hspace=-0.2)

g.set_titles("")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)

for ax in g.axes.flat:
    ax.axvline(x=t_df[value].median(), color='r', linestyle='-')


plt.show()

# Detección de dobletes

In [ ]:
import scanpy as sc
import scvi

# Iterar sobre las listas de AnnData en c_adatas
for i, adata in enumerate(l_adatas):
    print(f"Procesando el conjunto de datos {i+1}")
    
    # Paso 1: Preprocesamiento
    # Filtrar genes que no se expresan en al menos 10 células
    sc.pp.filter_genes(adata, min_cells=10)
    
    # Seleccionar los 2000 genes más variables
    sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=False, flavor='cell_ranger')
    sc.pp.pca(adata, n_comps=50, mask_var="highly_variable")
    sc.pp.neighbors(adata, use_rep='X_pca')
    sc.tl.umap(adata)
    # Verificación después del preprocesamiento
    print(f"Datos después del filtrado de genes y selección de genes variables: {adata.shape}")
    adata.X = adata.X.tocsr()
    # Paso 2: Configurar y entrenar el modelo scVI
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()

    # Paso 3: Aplicar SOLO para detectar dobletes
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()

    # Obtener las predicciones
    df = solo.predict()
    
    # Generar etiquetas binarias (predicción directa de doublet o singlet)
    df['prediction'] = solo.predict(soft=False)
    
    # Calcular la diferencia entre la probabilidad de 'doublet' y 'singlet'
    df['dif'] = df.doublet - df.singlet

    # Paso 4: Filtrar células dobletes con un umbral de dif > 0.9
    doublets = df[(df.prediction == 'doublet') & (df.dif > 0.9)]

    # Ver cuántas células están etiquetadas como dobletes
    print(f"Total de células etiquetadas como doblete con dif > 0.9: {doublets.shape[0]}")

    # Paso 5: Crear una nueva columna en adata.obs que indique si la célula es un doblete
    adata.obs['doublet'] = adata.obs.index.isin(doublets.index)

    # Paso 6: Filtrar las células que no son dobletes (i.e., mantener solo singletes)
    adata_filtrado = adata[~adata.obs['doublet']]

    # Actualizar la lista c_adatas con el objeto filtrado
    l_adatas[i] = adata_filtrado

    # Verificación después del filtrado de dobletes
    print(f"Datos restantes después de filtrar dobletes: {adata_filtrado.shape}")

#  Lectura de los nuevos archivos procesados
Normalización de los datos

In [3]:
def norm (adata):
    sc.pp.normalize_total(adata, target_sum=10**4)
    sc.pp.log1p(adata)
    return adata

In [4]:
l_adatas = [x for x in os.listdir('./filtered_files/') if x.endswith('.h5ad')]
l_adatas = [ sc.read_h5ad('./filtered_files/' + ad)  for ad in l_adatas]
# Normalizamos los datos
l_adatas = [norm(ad) for ad in l_adatas]

In [5]:
# Vamos a imprimir la información de los objetos
for adata in l_adatas:
    print(f"Después de normalizar: {adata.obs['Patient'][0]}_{adata.obs['DX'][0]}: {adata.n_obs} cells, {adata.n_vars} genes, total counts: {adata.X.sum()}")
    print(adata.obs['doublet'].value_counts())
    print(adata.obs['doublet'].value_counts(normalize=True))
    print(adata.obs['doublet'].value_counts(normalize=True).sum())
    print("-"*50)

/tmp/ipykernel_3785895/2717862215.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Después de normalizar: {adata.obs['Patient'][0]}_{adata.obs['DX'][0]}: {adata.n_obs} cells, {adata.n_vars} genes, total counts: {adata.X.sum()}")


Después de normalizar: GSM3489189_Donor: 9210 cells, 16019 genes, total counts: 15822775.0
doublet
False    9210
Name: count, dtype: int64
doublet
False    1.0
Name: proportion, dtype: float64
1.0
--------------------------------------------------
Después de normalizar: GSM3489197_Donor: 27618 cells, 16748 genes, total counts: 36933592.0
doublet
False    27618
Name: count, dtype: int64
doublet
False    1.0
Name: proportion, dtype: float64
1.0
--------------------------------------------------
Después de normalizar: GSM3489192_HP: 6180 cells, 17307 genes, total counts: 12813002.0
doublet
False    6180
Name: count, dtype: int64
doublet
False    1.0
Name: proportion, dtype: float64
1.0
--------------------------------------------------
Después de normalizar: GSM3489190_IPF: 5901 cells, 13538 genes, total counts: 8715935.0
doublet
False    5901
Name: count, dtype: int64
doublet
False    1.0
Name: proportion, dtype: float64
1.0
--------------------------------------------------
Después de n

# Integración de los 17 archivos normalizados